In [1]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [2]:
from PIL import Image
image = Image.open("cat.jpg")
inputs = processor(
    text=["a photo of a cat", "a photo of a dog"],
    images=image,
    return_tensors="pt",
    padding=True,
)

In [3]:
from video_processor import VideoProcessor

In [4]:
vp = VideoProcessor(
  clip_model.text_model,
  clip_model.vision_model,
  processor.tokenizer,
  processor.image_processor,
)

In [6]:
vp = vp.to('cuda:3')

In [7]:
import os
os.getcwd()
videos_dir = "/home/basha_2211ai03/complaint_detection/videos/dataset"
videos = os.listdir(videos_dir)

In [8]:
import torch
with torch.inference_mode():
    data = vp(os.path.join(videos_dir, "_A5qfpLTbns.mp4"))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Extracting 2 sec batches


  5%|▌         | 9/180 [01:05<18:49,  6.60s/it]

In [7]:
vision.shape

torch.Size([180, 768])

In [ ]:
out = vp.image_model(**processor.image_processor([image,image], return_tensors="pt").to("cuda"))

In [ ]:
out.pooler_output.mean(axis=0).shape

In [ ]:
processor.tokenizer('basha')

In [ ]:

out = clip_model.text_model(
    input_ids = inputs['input_ids'],
    attention_mask = inputs['attention_mask']
)

In [ ]:
out.last_hidden_state.shape

In [ ]:
out.pooler_output

In [ ]:
out.last_hidden_state[0]